# Lectura 38: LazyFrame - Miscelánea

## `map_batches`

Esta función permite aplicar una función personalizada a un LazyFrame. Es importante que la función personalizada devuelva un DataFrame de Polars.

In [ ]:
import polars as pl
from polars import col

vuelos_lz = (
    pl.scan_parquet('./data/vuelos/vuelos.parquet')
)

In [ ]:
(
    vuelos_lz.select(
        col('AIR_TIME'),
        col('DEPARTURE_DELAY')
    )
    .map_batches(lambda c: (c / 60).cast(pl.Int32))
    .collect()  
)   

## `pipe`

Esta función ofrece una forma estructurada de aplicar una secuencia de funciones definidas por el usuario (UDF).

In [ ]:
def fill_null_int_str(data, col_name):
    
    if str(data.select(col_name).dtypes[0]) == 'String':
        
        return data.fill_null('LOST')
    
    else:
        
        return data.fill_null(-999)

In [ ]:
vuelos_lz.pipe(fill_null_int_str, 'CANCELLATION_REASON', ).collect()

In [ ]:
vuelos_lz.pipe(fill_null_int_str, 'WEATHER_DELAY').collect()

## `profile`

Esta función perfila un LazyFrame, es decir, ejecutará la consulta y devolverá una tupla que contiene el DataFrame materializado y un DataFrame que contiene información de perfil de cada nodo que se ejecuta. Las unidades de los tiempos son microsegundos.

Con el parámetro `show_plot=True` mostraremos un diagrama de Gantt del resultado del perfilado.

In [ ]:
query_pf = vuelos_lz.group_by('AIRLINE').agg(
    SUM_AIR_TIME=col('AIR_TIME').sum()
).sort('SUM_AIR_TIME', descending=True)

query_pf.profile(show_plot=True)

## `serialize y deserialize`

Estas funciones nos permitirán serializar y deserializar el plan lógico de un LazyFrame.

In [ ]:
query_pf.serialize('./output.txt')

In [ ]:
import io

# Abrimos el archivo en modo binario
with open("output.txt", "rb") as file_object:
    
    # Leemos el contenido del archivo entero como bytes
    data = file_object.read()

# Creamos un objeto de tipo BytesIO a partir de los datos binarios
query = io.BytesIO(data)

pl.LazyFrame.deserialize(query).collect()